# Análise da Qualidade dos Dados

Nesta etapa, avaliamos a qualidade dos dados da camada **Gold** do projeto de análise do mercado de short-term rentals (Airbnb) para **Rio de Janeiro** e **New York City**.

O objetivo é verificar se as tableas dimensionais e fato:
- estão completas nos campos críticos;
- respeitam a granularidade definida na modelagem;
- mantêm a integridade referencial entre si;
- apresentam valores dentro de faixas plausíveis (preço, capacidade, notas, etc.).

Os testes de qualidade serão executados diretamente sobre as tabelas do catálogo:
'mvp_airbnb_catalog.default'

Tabelas análisadas:
- **Dimensões**
  - `dim_listings`
  - `dim_hosts`
  - `dim_neighbourhoods`
- **Fatos**
  - `fact_calenda`
  - `fact_reviews`



In [0]:
from pyspark.sql import functions as F

# Verificar se o catálogo e o schema do MVP estão acessíveis
spark.sql("SHOW TABLES IN mvp_airbnb_catalog.default").show(truncate=False)

## Verificação inicial do catálogo e das tabelas Gold

Antes de iniciar os testes de qualidade, foi verificado se o catálogo e o schema do projeto (`mvp_airbnb_catalog.default`) estavam acessíveis a partir do notebook Python no Databricks.

O comando acima (`SHOW TABLES`) confirmou a existência das tabelas criadas nas etapas anteriores (Bronze, Silver e Gold), incluindo:

- `dim_listings`
- `dim_hosts`
- `dim_neighbourhoods`
- `fact_calendar`
- `fact_reviews`

Isso garante que a camada Gold está devidamente registrada no **Unity Catalog** e pronta para ser utilizada nos testes de qualidade.


## Testes de Completude

A completude avalia se os campos considerados **críticos para análise** estão preenchidos, isto é, se não há uma quantidade significativa de valores nulos que comprometa a interpretação dos resultados.

Começamos pela tabela central do modelo dimensional:

### Completude - `dim_listings`

Campos avaliados:

- `listing_id` - identificador único do anúncio
- `host_id` - identificador do anfitrião
- `neighbourhood_cleansed` - bairro associado ao anúncio
- `room_type` - tipo de acomodação (entire home, private room, etc.)
- `accommodates` - capacidade máxima de hóspedes
- `price_numeric` - preço numérico do anúncio

A consulta abaixo contabiliza o total de linhas da dimensão e quantos registros possuem `NULL` em cada um desses campos.


In [0]:
df_nulls_dim_listings = spark.sql("""
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN listing_id IS NULL THEN 1 ELSE 0 END) AS null_listing_id,
  SUM(CASE WHEN host_id IS NULL THEN 1 ELSE 0 END) AS null_host_id,
  SUM(CASE WHEN neighbourhood_cleansed IS NULL THEN 1 ELSE 0 END) AS null_neighbourhood,
  SUM(CASE WHEN room_type IS NULL THEN 1 ELSE 0 END) AS null_room_type,
  SUM(CASE WHEN accommodates IS NULL THEN 1 ELSE 0 END) AS null_accommodates,
  SUM(CASE WHEN price_numeric IS NULL THEN 1 ELSE 0 END) AS null_price
FROM mvp_airbnb_catalog.default.dim_listings
""")

df_nulls_dim_listings.show(truncate=False)


In [0]:
df = df_nulls_dim_listings.toPandas()
row = df.iloc[0]
total = row["total"]

def pct(x):
    return round(100 * x / total, 2) if total > 0 else 0

details_dim_listings = f"""
Total de registros em dim_listings: {total}

- listing_id NULL: {row['null_listing_id']} ({pct(row['null_listing_id'])}%)
- host_id NULL: {row['null_host_id']} ({pct(row['null_host_id'])}%)
- neighbourhood_cleansed NULL: {row['null_neighbourhood']} ({pct(row['null_neighbourhood'])}%)
- room_type NULL: {row['null_room_type']} ({pct(row['null_room_type'])}%)
- accommodates NULL: {row['null_accommodates']} ({pct(row['null_accommodates'])}%)
- price_numeric NULL: {row['null_price']} ({pct(row['null_price'])}%)
"""

print(details_dim_listings)


### Resultados - Completude em `dim_listings`

**Resumo dos resultados:**

- Total de registros: **79.179**
- `listing_id` nulo: **0**
- `host_id` nulo: **0**
- `neighbourhood_cleansed` nulo: **0**
- `room_type` nulo: **0**
- `accommodates` nulo: **0**
- `price_numeric` nulo: **19.181 (24,22%)**

### **Interpretação**

A tabela `dim_listings` apresenta excelente completude nos campos estruturais:

- As chaves (`listing_id`, `host_id`) não possuem valores nulos, garantindo integridade e confiabilidade no relacionamento com tabelas fato.
- Os atributos categóricos e de localização (`neighbourhood_cleansed`, `room_type`) também estão 100% completos, o que viabiliza análises geográficas e por tipo de acomodação sem prejuízo.

O único campo crítico com volume significativo de valores ausentes é:

- **`price_numeric`**, com **24,22%** de registros sem preço.

Algumas hipóteses que podem explicar registros sem preço:
- alguns anúncios podem estar offline no momento da coleta,
- podem esconder o preço,
- ou podem estar cadastrados sem valor definido,
- Alguns anúncios têm informações operacionais incompletas

**Impacto Analítico:**

- Afeta análises que dependem diretamente de preço (como comparação de diárias e estimativa de revenue).
- Não compromete análises estruturais (quantidade de anúncios, tipos, distribuição geográfica).
- Na camada Gold, esses registros são mantidos por rastreabilidade, mas serão filtrados quando métricas de preço forem calculadas.



### Teste 1 - Completude de `price_numeric` e Presença no Calendário

#### Objetivo do teste

Verificar se os anúncios com preço nulo (`price_numeric IS NULL`) na tabela `dim_listings` correspondem a:

- anúncios desativados (sem registros no calendário), ou  
- anúncios ativos com calendário publicado, mas sem preço explícito registrado na dimensão.

Este teste responde à pergunta:

> Os anúncios com `price_numeric` nulo estão ativos no marketplace (ou seja, aparecem na tabela de calendário)?


In [0]:
%sql
SELECT
    COUNT(DISTINCT dl.listing_id) AS total_null_price_listings,
    COUNT(DISTINCT CASE WHEN fc.listing_id IS NULL THEN dl.listing_id END) AS listings_missing_in_calendar,
    COUNT(DISTINCT CASE WHEN fc.listing_id IS NOT NULL THEN dl.listing_id END) AS listings_present_in_calendar
FROM mvp_airbnb_catalog.default.dim_listings dl
LEFT JOIN mvp_airbnb_catalog.default.fact_calendar fc
    ON dl.city = fc.city
   AND dl.listing_id = fc.listing_id
WHERE dl.price_numeric IS NULL;


#### Resultados do Teste 1 - Completude de `price_numeric` e Presença no Calendário

- **Anúncios com `price_numeric` nulo:** 19.181  
- **Listings ausentes no calendário:** 0  
- **Listings presentes no calendário:** 19.181  

#### Interpretação

Todos os anúncios sem preço aparecem na tabela `fact_calendar`, indicando que:

- nenhum deles está desativado ou removido da plataforma;  
- todos possuem calendário publicado e, portanto, fazem parte do inventário ativo;  
- a ausência de preço não está relacionada a inatividade do anúncio.

Esse comportamento é compatível com situações em que:

- o host oculta o preço até que o usuário selecione datas,  
- o anúncio utiliza precificação dinâmica,  
- a tarifa base não está registrada de forma explícita no dataset público,  
- o anúncio está parcialmente configurado.

O teste confirma que a ausência de preço não compromete a integridade do modelo de dados, mas exige tratamento específico nas análises que dependem de valor monetário.


### Teste 2 - Atividade Recente via Reviews (últimos 180 dias)

#### Objetivo do teste

Investigar se os anúncios com `price_numeric` nulo continuam ativos do ponto de vista operacional, analisando a data dos reviews recebidos.

A lógica é:

- **Se um anúncio recebeu reviews nos últimos 180 dias**, ele está ativo e recebendo hóspedes, mesmo que o preço base não esteja registrado.
- **Se não recebeu reviews recentes**, ele pode estar ativo mas com baixa demanda, ou pode ser um anúncio recente sem histórico.

Este teste responde à pergunta:

> Entre os anúncios sem preço, quantos continuam recebendo hóspedes (reviews recentes) e quantos não possuem atividade recente?



In [0]:
%sql
SELECT
    COUNT(*) AS total_null_price,
    SUM(CASE WHEN last_review_date >= date_sub(current_date(), 180)
             THEN 1 ELSE 0 END) AS active_last_6m,
    SUM(CASE WHEN last_review_date < date_sub(current_date(), 180)
             OR last_review_date IS NULL THEN 1 ELSE 0 END) AS inactive_or_no_reviews
FROM mvp_airbnb_catalog.default.dim_listings
WHERE price_numeric IS NULL;


#### Resultados do Teste 2 - Atividade Recente via Reviews (últimos 180 dias)

- **Anúncios com `price_numeric` nulo:** 19.181  
- **Com reviews nos últimos 180 dias (ativos):** 1.636  
- **Sem reviews recentes ou sem reviews (inativos/baixa atividade):** 17.545  

#### Interpretação

A análise mostra que, entre os anúncios sem preço explícito:

- **Cerca de 8,52% (1.636 listings)** continuam recebendo hóspedes, conforme evidenciado por reviews recentes.  
  Isso indica que:
  - esses anúncios estão efetivamente ativos no marketplace,  
  - o preço pode estar oculto, dinâmico ou não capturado pelo dataset,  
  - o anúncio segue operando normalmente na plataforma.

- **A grande maioria (91,48% / 17.545 listings)** não possui reviews recentes ou não possui reviews registradas.  
  Isso sugere três possíveis situações:
  1. **Anúncios novos**, recém-cadastrados e ainda sem histórico de hospedagem;  
  2. **Anúncios ativos com baixa demanda**, que não receberam hóspedes no período analisado;  
  3. **Anúncios antigos e pouco utilizados**, com presença no calendário mas baixa atividade operacional.

Esse comportamento é coerente com marketplaces de aluguel de temporada, em que muitos anúncios permanecem online com baixa ocupação ou ainda em fase inicial de operação.  

O teste reforça a conclusão do Teste 1:  
> A ausência de preço não se relaciona à inatividade sistêmica dos anúncios - trata-se principalmente de anúncios com baixa atividade ou com configuração incompleta de preço.


### Teste 3 - Ausência de Reviews (Indicador de Novos Anúncios ou Baixa Demanda)

#### Objetivo do teste

Determinar, entre os anúncios com `price_numeric` nulo, quantos **não possuem qualquer review registrado**.  
A ausência total de reviews pode indicar que:

- o anúncio é **novo**, ainda sem histórico de hospedagens;
- o anúncio está **ativo**, mas com **baixa demanda**, sem receber hóspedes;
- o dataset não capturou reviews (situação rara, mas possível);
- o host não acumulou avaliações suficientes para que o InsideAirbnb registre uma data de review.

Este teste responde à pergunta:

> Entre os anúncios sem preço, quantos nunca receberam reviews e podem ser considerados novos ou sem atividade histórica?


In [0]:
%sql
SELECT
    COUNT(*) AS total_null_price,
    SUM(CASE WHEN last_review_date IS NULL THEN 1 ELSE 0 END) AS no_reviews,
    SUM(CASE WHEN reviews_per_month IS NULL OR reviews_per_month = 0 THEN 1 ELSE 0 END) AS zero_review_frequency
FROM mvp_airbnb_catalog.default.dim_listings
WHERE price_numeric IS NULL;


#### Resultados do Teste 3 - Ausência de Reviews (novos anúncios ou baixa demanda)

- **Anúncios com `price_numeric` nulo:** 19.181  
- **Listings sem qualquer review registrado (`last_review_date IS NULL`):** 5.909  
- **Listings com frequência de reviews igual a zero (`reviews_per_month = 0`):** 5.909  

#### Interpretação

Entre os anúncios sem preço explícito:

- **5.909 listings (30,8%) nunca receberam avaliações.**  
  Esse é um indicador muito forte de que esses anúncios se enquadram em um dos seguintes cenários:
  1. **Anúncios novos**, recém-criados e sem histórico operacional;  
  2. **Anúncios ativos porém sem demanda**, isto é, disponíveis para reserva mas sem hóspedes nos últimos meses;  
  3. Listagens que aparecem no inventário do Airbnb, mas ainda não capturaram interação suficiente para gerar avaliações.

- O fato de `no_reviews` e `zero_review_frequency` apresentarem exatamente o mesmo valor reforça que:
  - As estimativas de frequência de reviews são coerentes com o histórico de reviews;  
  - Listings com `last_review_date = NULL` também apresentam zero reviews por mês, como esperado.

#### Conclusão do Teste 3

O Teste 3 confirma que uma parcela substancial dos anúncios sem preço - **cerca de 31%** - são anúncios **sem histórico de hóspedes**, seja por serem novos ou por terem baixa atratividade no marketplace.

Esse resultado complementa o Teste 2:

- **1.636 listings** têm reviews recentes (ativos)  
- **5.909 listings** nunca tiveram reviews (novos / baixa demanda)  
- **11.636 listings** têm reviews antigos, indicando **atividade passada, porém não recente**

Esse padrão é típico em mercados com alto turnover de anúncios, como o Airbnb, e deve ser levado em conta na construção das métricas de precificação e rentabilidade na camada Gold.


### Teste 4 - Consistência Operacional via Estimativas de Ocupação e Receita

#### Objetivo do teste

Avaliar, entre os anúncios com `price_numeric` nulo, quantos possuem estimativas operacionais calculadas pelo InsideAirbnb:

- `estimated_occupancy_l365d` → estimativa da ocupação nos últimos 365 dias  
- `estimated_revenue_l365d` → estimativa de receita anual

Essas colunas são derivadas de modelos do próprio InsideAirbnb e refletem a capacidade de calcular métricas a partir do histórico do anúncio.

Este teste responde à pergunta:

> Entre os anúncios sem preço, quantos têm dados suficientes para gerar estimativas de ocupação e receita, e quantos carecem de informações para análises de performance?


In [0]:
%sql
SELECT
    COUNT(*) AS total_null_price,
    SUM(CASE WHEN estimated_occupancy_l365d IS NULL THEN 1 ELSE 0 END) AS null_estimated_occupancy,
    SUM(CASE WHEN estimated_revenue_l365d IS NULL THEN 1 ELSE 0 END) AS null_estimated_revenue
FROM mvp_airbnb_catalog.default.dim_listings
WHERE price_numeric IS NULL;


#### Resultados do Teste 4 - Consistência Operacional via Estimativas

- **Anúncios com `price_numeric` nulo:** 19.181  
- **Listings sem estimativa de ocupação (`estimated_occupancy_l365d` nulo):** 0  
- **Listings sem estimativa de receita (`estimated_revenue_l365d` nulo):** 19.181  

#### Interpretação

O comportamento das duas variáveis é bastante distinto:

#####  `estimated_occupancy_l365d` - Completude total  
Nenhum dos 19.181 anúncios apresenta valor nulo para `estimated_occupancy_l365d`.  
Isso indica que:

- O InsideAirbnb conseguiu calcular uma estimativa de ocupação para **todos** os listings, mesmo quando o preço não está disponível.
- A ocupação estimada é derivada principalmente da **disponibilidade histórica** (calendário), e não do preço.

Esse dado reforça:
- os listings estão ativos no calendário,  
- há informação suficiente (disponibilidade) para o cálculo de ocupação.

#####  `estimated_revenue_l365d` - Completamente nulo  
Todos os anúncios sem preço (19.181) apresentam `estimated_revenue_l365d` nulo.  
Isso confirma que:

- a receita estimada depende **diretamente da presença de um preço válido**;  
- se o preço não está disponível, o modelo do InsideAirbnb **não consegue inferir receita anual**;  
- listings sem preço formam automaticamente um *subconjunto improdutivo para estimativas de receita real*.

Essa relação é coerente com o comportamento esperado:

> Sem preço → sem estimativa de receita.

#### Conclusão do Teste 4

O Teste 4 reforça as conclusões dos testes anteriores:

- Os anúncios sem preço **não são inativos** - todos possuem estimativa de ocupação.  
- A estimativa de ocupação é confiável e utilizável na análise, pois está presente em 100% dos casos.  
- A ausência de preço inviabiliza qualquer métrica de receita histórica, exigindo:
  - exclusão desses anúncios das análises de rentabilidade, ou  
  - criação de estratégias de imputação (caso a modelagem ou análise requeira).

Esses achados serão incorporados ao relatório final de qualidade e às decisões de modelagem analítica na camada Gold.


### Teste em `dim_hosts` - Completude e Consistência dos Atributos do Host

#### Objetivo do teste

Verificar a completude e consistência dos principais atributos da dimensão `dim_hosts`, avaliando:

- presença do identificador do host (`host_id`);
- completude de atributos essenciais (nome, data desde quando hospeda);
- indicadores operacionais (superhost, número de listings por host);
- classificação de perfil (`host_type`: individual vs. profissional).

Este teste responde à pergunta:

> A dimensão `dim_hosts` possui dados completos e consistentes para permitir análises de comportamento e perfil dos anfitriões?



In [0]:
%sql
SELECT
    COUNT(*) AS total_hosts,
    SUM(CASE WHEN host_id IS NULL THEN 1 END) AS null_host_id,
    SUM(CASE WHEN host_name IS NULL THEN 1 END) AS null_host_name,
    SUM(CASE WHEN host_since IS NULL THEN 1 END) AS null_host_since,
    SUM(CASE WHEN host_is_superhost_bool IS NULL THEN 1 END) AS null_superhost_flag,
    SUM(CASE WHEN host_total_listings_count IS NULL THEN 1 END) AS null_host_total_listings_count,
    SUM(CASE WHEN calculated_host_listings_count IS NULL THEN 1 END) AS null_calculated_listings_count,
    SUM(CASE WHEN host_type IS NULL THEN 1 END) AS null_host_type
FROM mvp_airbnb_catalog.default.dim_hosts;


#### Resultados do Teste em `dim_hosts` - Completude e Consistência

- **Total de hosts:** 46.375  
- **host_id nulo:** 0  
- **host_name nulo:** 25  
- **host_since nulo:** 26  
- **host_is_superhost_bool nulo:** 771  
- **host_total_listings_count nulo:** 26  
- **calculated_host_listings_count nulo:** 0  
- **host_type nulo:** 0  

#### Interpretação

Os resultados indicam que a dimensão `dim_hosts` apresenta **alta completude e consistência geral**, com poucas exceções:

##### - Integração de chaves e atributos estruturais
- A chave primária (`host_id`) está **100% completa**, garantindo integridade referencial com `dim_listings`.
- Atributos básicos como `host_name` e `host_since` apresentam baixíssima taxa de nulos (≈0,05%), não comprometendo análises.

##### - Indicadores operacionais
- O campo `host_is_superhost_bool` apresenta **771 valores nulos (1,66%)**, que podem ocorrer por:
  - ausência de dados no dataset original,
  - perfis antigos que não exibem esta classificação,
  - hosts com atividade limitada,
  - limitação de scraping da fonte.

Essa lacuna é pequena, mas deve ser tratada com cuidado em análises segmentadas.

##### - Métricas Gold e Classificação de Perfil
- `calculated_host_listings_count` está **100% completo**, validando a agregação criada na camada Gold.
- `host_type` (individual vs. profissional) também está **100% preenchido**, permitindo análises confiáveis de comportamento entre tipos de hosts.

#### Conclusão do Teste

A dimensão `dim_hosts` apresenta:

- **estrutura sólida**,  
- **completude elevada**,  
- **pouquíssimas lacunas**,  
- **classificação consistente**,  
- e **plena integridade referencial** com as demais dimensões e fatos.

As pequenas quantidades de nulos não prejudicam a análise e podem ser tratadas como casos isolados.  

Essa dimensão está pronta para suportar análises de:

- comportamento de hosts,  
- comparação entre profissionais vs. individuais,  
- impacto de superhost na performance,  
- distribuição de número de listings por host.

### Teste 5 - Integridade Referencial entre `dim_listings` e `dim_hosts`

#### Objetivo do teste

Verificar se todos os anfitriões (`host_id`) referenciados em `dim_listings` possuem um registro correspondente em `dim_hosts`, garantindo a integridade referencial do modelo.

Este teste responde à pergunta:

> Existe algum `host_id` utilizado em `dim_listings` que não esteja cadastrado em `dim_hosts`?


In [0]:
%sql
SELECT
    COUNT(DISTINCT dl.host_id) AS total_host_ids_in_listings,
    COUNT(DISTINCT h.host_id) AS matched_hosts,
    COUNT(DISTINCT CASE WHEN h.host_id IS NULL THEN dl.host_id END) AS missing_hosts
FROM mvp_airbnb_catalog.default.dim_listings dl
LEFT JOIN mvp_airbnb_catalog.default.dim_hosts h
    ON dl.city = h.city
   AND dl.host_id = h.host_id;


#### Resultados do Teste 5 - Integridade Referencial entre `dim_listings` e `dim_hosts`

- **Hosts distintos referenciados em `dim_listings`:** 46.366  
- **Hosts encontrados em `dim_hosts`:** 46.366  
- **Hosts ausentes (órfãos):** 0  

#### Interpretação

O resultado demonstra integridade referencial completa entre as duas tabelas:

- Todo `host_id` presente em `dim_listings` possui um registro correspondente em `dim_hosts`.  
- Isso significa que **não existem listings associados a hosts inexistentes**, reforçando a consistência do processo de ingestão e modelagem.
- A relação Host → Listings está completamente representada na modelagem, sem lacunas.

Esse comportamento é desejado em um modelo dimensional, garantindo:

- confiabilidade nas análises de perfil e comportamento de anfitriões,  
- consistência ao cruzar métricas de listings com características de hosts,  
- ausência de registros isolados que poderiam distorcer métricas de performance.

#### Conclusão do Teste

> A integridade referencial entre `dim_listings` e `dim_hosts` está **100% preservada**, validando a robustez do pipeline de ingestão e da construção das dimensões.

Essa validação é um ponto forte na etapa de qualidade dos dados, pois garante que o modelo Gold representa adequadamente as entidades do marketplace.


### Teste em `dim_neighbourhoods` - Completude e Consistência dos Bairros

#### Objetivo do teste

Avaliar a qualidade da dimensão `dim_neighbourhoods`, verificando:

- se todos os bairros (`neighbourhood`) estão preenchidos;
- se há preenchimento consistente de `neighbourhood_group` (quando aplicável);
- se o campo geoespacial (`geometry_json`) está presente;
- se há variação adequada por cidade (Rio de Janeiro x New York City).

Este teste responde às perguntas:

> A dimensão de bairros está completa e consistente o suficiente para análises geográficas?  
> Existem bairros sem geometria ou sem grupo associado?


In [0]:
%sql
SELECT
    city,
    COUNT(*) AS total_neighbourhoods,
    SUM(CASE WHEN neighbourhood IS NULL THEN 1 END) AS null_neighbourhood,
    SUM(CASE WHEN neighbourhood_group IS NULL THEN 1 END) AS null_neighbourhood_group,
    SUM(CASE WHEN geometry_json IS NULL THEN 1 END) AS null_geometry_json
FROM mvp_airbnb_catalog.default.dim_neighbourhoods
GROUP BY city;


#### Resultados do Teste em `dim_neighbourhoods` - Completude e Consistência

- Rio de Janeiro
  - Total de bairros: 160
  - Bairros nulos: 0
  - Grupo de Bairro nulos: 160
  - Nulos na coluna geometry_json: 0
- NYC
  - Total de bairros: 233
  - Bairros nulos: 0
  - Grupo de Bairros nulos: 0
  - Nulos na coluna geometry_json: 0

#### Interpretação

Os resultados confirmam que:

##### `neighbourhood` está 100% completo
Nenhuma cidade possui bairros com nome ausente, garantindo integridade básica da dimensão.

##### `geometry_json` está 100% completo
Após o reparo aplicado na ingestão (conversão GeoJSON → CSV corrigida), todos os bairros possuem geometria válida:

- Permite construir mapas e análises espaciais;
- Confirma que a etapa de ETL funcionou corretamente;
- Demonstra consistência geográfica na dimensão.

##### `neighbourhood_group` é **sempre nulo no Rio**, mas completo em NYC
Isso reflete **características reais do dataset**, não um problema:

- Nova York possui *neighbourhood groups* ("Manhattan", "Brooklyn", etc.).
- O Rio de Janeiro **não utiliza essa classificação**, por isso 160 valores nulos são esperados e corretos.

Essa diferença será documentada no relatório final como **heterogeneidade natural entre cidades**.

#### Conclusão do Teste

> A dimensão de bairros apresenta alta completude e consistência, com nomes e geometrias totalmente preenchidos.  
> A ausência de `neighbourhood_group` no Rio é esperada e corresponde à estrutura original do dataset InsideAirbnb.

A dimensão está pronta para suportar análises geográficas, comparações entre cidades e relacionamento com `dim_listings`.


### Teste 6 - Integridade Referencial entre `dim_listings` e `dim_neighbourhoods`

#### Objetivo do teste

Verificar se todos os anúncios (`listing_id`) em `dim_listings` estão associados a um bairro válido cadastrado em `dim_neighbourhoods`, respeitando a combinação (`city`, `neighbourhood`).

Este teste responde à pergunta:

> Existe algum anúncio em `dim_listings` com `neighbourhood_cleansed` que não esteja cadastrado em `dim_neighbourhoods`?


In [0]:
%sql
SELECT
    COUNT(DISTINCT dl.listing_id) AS total_listings,
    COUNT(DISTINCT CASE WHEN n.neighbourhood IS NOT NULL THEN dl.listing_id END) AS listings_with_valid_neighbourhood,
    COUNT(DISTINCT CASE WHEN n.neighbourhood IS NULL THEN dl.listing_id END) AS listings_with_missing_neighbourhood
FROM mvp_airbnb_catalog.default.dim_listings dl
LEFT JOIN mvp_airbnb_catalog.default.dim_neighbourhoods n
    ON dl.city = n.city
   AND dl.neighbourhood_cleansed = n.neighbourhood;


#### Resultados do Teste 6 - Integridade Referencial entre `dim_listings` e `dim_neighbourhoods`

- **Total de listings:** 79.179  
- **Listings com bairro válido na dimensão:** 79.179  
- **Listings com bairro sem correspondência (órfãos):** 0  

#### Interpretação

O modelo apresenta **integridade referencial completa** entre `dim_listings` e `dim_neighbourhoods`, garantindo:

- todos os anúncios estão vinculados a um bairro cadastrado,  
- não existem bairros “fantasma” na dimensão,  
- o relacionamento geográfico Cidade → Bairro → Listing funciona perfeitamente,  
- não há risco de análises espaciais inconsistentes.

Essa consistência reforça que:

- o processo de ingestão (ETL) limpou corretamente os nomes dos bairros;  
- os datasets originais estavam bem alinhados;  
- não foi necessário aplicar normalização extra na camada Silver.

#### Conclusão do Teste

> A integridade referencial entre anúncios e bairros está 100% preservada.  
> A dimensão `dim_neighbourhoods` está pronta para análises geográficas, clusterização espacial, comparações entre cidades e enriquecimento de métricas da camada Gold.

Este é um dos pilares mais importantes da modelagem espacial do MVP, e o resultado confirma a robustez do pipeline.


A tabela `fact_calendar` representa a granularidade diária dos anúncios, contendo informações de:

- disponibilidade,
- ocupação,
- preços diários,
- requisitos mínimos/máximos de estadia,
- receita estimada.

Antes de utilizá-la em análises Gold, é essencial validar sua integridade, ranges, nulos e consistência com `dim_listings`.

### Teste 1 - Completude dos Campos Principais

#### Objetivo do teste

Avaliar se os campos fundamentais da tabela `fact_calendar` apresentam valores nulos:

- `listing_id`
- `date_date`
- `available_bool`
- `is_occupied`
- `price_numeric`
- `adjusted_price_numeric`
- `minimum_nights`
- `maximum_nights`
- `revenue_estimated`

Este teste responde à pergunta:

> A tabela `fact_calendar` está completa o suficiente para análises de disponibilidade, preço e receita?


In [0]:
%sql
SELECT
    COUNT(*) AS total_rows,
    SUM(CASE WHEN listing_id IS NULL THEN 1 END) AS null_listing_id,
    SUM(CASE WHEN date_date IS NULL THEN 1 END) AS null_date,
    SUM(CASE WHEN available_bool IS NULL THEN 1 END) AS null_available,
    SUM(CASE WHEN is_occupied IS NULL THEN 1 END) AS null_is_occupied,
    SUM(CASE WHEN price_numeric IS NULL THEN 1 END) AS null_price,
    SUM(CASE WHEN adjusted_price_numeric IS NULL THEN 1 END) AS null_adjusted_price,
    SUM(CASE WHEN minimum_nights IS NULL THEN 1 END) AS null_min_nights,
    SUM(CASE WHEN maximum_nights IS NULL THEN 1 END) AS null_max_nights,
    SUM(CASE WHEN revenue_estimated IS NULL THEN 1 END) AS null_revenue
FROM mvp_airbnb_catalog.default.fact_calendar;


### Teste 1 - Completude dos Campos Principais em `fact_calendar`

#### Resultados

| Métrica                     | Valor        |
|-----------------------------|--------------|
| Total de linhas             | 28,900,362   |
| listing_id nulo             | 0            |
| date_date nulo              | 0            |
| available_bool nulo         | 0            |
| is_occupied nulo            | 0            |
| price_numeric nulo          | 28,900,362   |
| adjusted_price_numeric nulo | 28,900,362   |
| minimum_nights nulo         | 0            |
| maximum_nights nulo         | 0            |
| revenue_estimated nulo      | 0            |

#### Interpretação

Os resultados mostram que:

- Os campos estruturais essenciais (`listing_id`, `date_date`, `available_bool`, `is_occupied`, `minimum_nights`, `maximum_nights`) **estão completos**, sem valores nulos.
- Os campos de **preço diário** (`price_numeric` e `adjusted_price_numeric`) estão **100% nulos**.
- Apesar disso, o campo `revenue_estimated` **não é nulo**, pois foi calculado usando o preço médio da dimensão `dim_listings`.

Isto confirma o comportamento já observado na fase Silver:

- O dataset do Inside Airbnb **não fornece preço diário** para Rio de Janeiro nem para NYC.
- Portanto, nossa `fact_calendar` cumpre papel analítico apenas para **ocupação**, enquanto os cálculos monetários dependem de valores derivados da `dim_listings`.

#### Conclusão do Teste

A tabela `fact_calendar` possui completude total nos campos necessários para:

- análise de ocupação,
- sazonalidade,
- disponibilidade,
- restrições de noites mínimas/máximas.

Já os campos de preços diários estão nulos por design do dataset, e não por erro de ingestão.  
Essa característica será documentada na seção de limitações do MVP, e será considerada na camada Gold durante a estimativa de receita.



### Teste 2 - Coerência Lógica entre `available_bool` e `is_occupied`

#### Objetivo do teste

Validar se o relacionamento lógico entre os campos:

- `available_bool`  
- `is_occupied`

foi mantido corretamente após a transformação da Silver para a Gold.

As regras de negócio esperadas são:

- **Se `available_bool = TRUE`, então o anúncio deve estar desocupado → `is_occupied = FALSE`**  
- **Se `available_bool = FALSE`, então o anúncio deve estar ocupado → `is_occupied = TRUE`**

Esse teste serve para identificar:

- inversões de disponibilidade,
- erros de transformação na Silver ou Gold,
- inconsistências no dataset de origem,
- registros logicamente impossíveis (como um anúncio marcado como disponível e ocupado ao mesmo tempo).

#### Pergunta respondida pelo teste

> As colunas `available_bool` e `is_occupied` estão coerentes entre si, sem contradições lógicas?

Uma incoerência aqui indicaria erros em:

- engenharia de dados (transformação da tabela),
- entendimento dos flags de ocupação do dataset,
- interpretação incorreta das regras de negócio.

#### Procedimento

O teste calcula:

- quantos registros violam a regra “disponível → não ocupado”  
- quantos registros violam a regra “indisponível → ocupado”  

Se ambos forem iguais a **zero**, a modelagem está consistente.



In [0]:
%sql
SELECT
    COUNT(*) AS total_rows,
    SUM(CASE WHEN available_bool = TRUE  AND is_occupied = TRUE  THEN 1 END) AS contradictions_available_and_occupied,
    SUM(CASE WHEN available_bool = FALSE AND is_occupied = FALSE THEN 1 END) AS contradictions_unavailable_and_not_occupied
FROM mvp_airbnb_catalog.default.fact_calendar;


#### Resultados do Teste 2 - Coerência Lógica entre `available_bool` e `is_occupied`

| Métrica                                         | Valor      |
|-------------------------------------------------|------------|
| Total de registros analisados                   | 28.900.362 |
| available = TRUE e is_occupied = TRUE (erro)   | 0          |
| available = FALSE e is_occupied = FALSE (erro) | 0          |

#### Interpretação

Os resultados mostram **coerência perfeita** entre os campos:

- `available_bool`  
- `is_occupied`

Não foram encontradas combinações inválidas. Isso significa que:

- a transformação feita na camada Silver respeitou corretamente a regra de negócio,
- a ingestão original do dataset está consistente,
- a camada Gold preservou a lógica sem introduzir erros.

Esse é um indicador forte de que:

- **o dataset calendar está confiável para análises de disponibilidade e ocupação**,  
- **não há necessidade de retrabalho estrutural nessa parte da modelagem**.

Essa validação é fundamental porque análises de ocupação, sazonalidade e receita dependem diretamente da integridade dessa relação lógica.


## Teste 1 - Completude dos Campos Essenciais em `fact_reviews`

### Objetivo do teste

Verificar se os campos fundamentais da tabela `fact_reviews` apresentam valores nulos.  
Esses campos são essenciais porque determinam:

- o relacionamento correto com `dim_listings` (`listing_id`),  
- a identificação única do review (`review_id`),  
- o posicionamento temporal da avaliação (`review_date`),  
- a disponibilidade de análise qualitativa (`comments_clean`).

Este teste responde à pergunta:

> A tabela `fact_reviews` possui dados completos o suficiente para análises de atividade, recência de reviews e insights qualitativos?

### Como o teste será realizado

Computaremos a contagem total de registros e a contagem de nulos nos seguintes campos:

- `listing_id`
- `review_id`
- `review_date`
- `comments_clean`

Após isso, interpretaremos a qualidade e possíveis implicações.


In [0]:
%sql
SELECT
  COUNT(*) AS total_reviews,
  SUM(CASE WHEN listing_id IS NULL THEN 1 END) AS null_listing_id,
  SUM(CASE WHEN review_id IS NULL THEN 1 END) AS null_review_id,
  SUM(CASE WHEN review_date IS NULL THEN 1 END) AS null_review_date,
  SUM(CASE WHEN comments_clean IS NULL THEN 1 END) AS null_comments
FROM mvp_airbnb_catalog.default.fact_reviews;

### Resultados do Teste 1 - Completude dos Campos Essenciais em `fact_reviews`

| Métrica                | Valor      |
|------------------------|------------|
| Total de reviews       | 2.114.313  |
| `listing_id` nulo      | 0          |
| `review_id` nulo       | 0          |
| `review_date` nulo     | 0          |
| `comments_clean` nulo  | 0          |

### Interpretação

Os resultados mostram **excelente qualidade de dados** para a tabela `fact_reviews`:

- Não há registros sem `listing_id` → **integridade referencial preservada**
- Todos os reviews possuem `review_id` → identificadores únicos completos
- Todas as avaliações possuem uma data → **cronologia confiável**
- Todos os reviews contêm `comments_clean` → dados textuais utilizáveis para análises qualitativas

Isso indica que:

- A ingestão das tabelas de reviews (Bronze → Silver → Gold) ocorreu sem perda de dados.  
- A limpeza textual (`comments_clean`) foi aplicada de maneira consistente.  
- A fact está apropriada para cálculos de:
  - recência de reviews,
  - frequência e sazonalidade de avaliações,
  - impacto de reviews no desempenho,
  - análises qualitativas caso desejado.

Essa é uma das tabelas com **melhor qualidade geral** em todo o modelo, sem necessidade de retrabalho.



## Teste 2 - Integridade Referencial entre `fact_reviews` e `dim_listings`

### Objetivo do teste

Verificar se todos os reviews registrados em `fact_reviews` estão associados a anúncios válidos em `dim_listings`, garantindo que:

- não existam reviews “órfãos” (sem anúncio correspondente);  
- o relacionamento `fact_reviews.listing_id` → `dim_listings.listing_id` esteja íntegro para ambas as cidades.

Este teste responde à pergunta:

> Todos os `listing_id` presentes em `fact_reviews` possuem um cadastro correspondente em `dim_listings`?


In [0]:
%sql
SELECT
    COUNT(DISTINCT fr.listing_id) AS listings_with_reviews_in_fact,
    COUNT(DISTINCT dl.listing_id) AS matched_listings_in_dim,
    COUNT(DISTINCT CASE WHEN dl.listing_id IS NULL THEN fr.listing_id END) AS missing_listings_in_dim
FROM mvp_airbnb_catalog.default.fact_reviews fr
LEFT JOIN mvp_airbnb_catalog.default.dim_listings dl
    ON fr.city = dl.city
   AND fr.listing_id = dl.listing_id;


### Resultados do Teste 2 - Integridade Referencial entre `fact_reviews` e `dim_listings`

| Métrica                                   | Valor   |
|-------------------------------------------|---------|
| Listings com reviews em `fact_reviews`    | 58.805  |
| Listings correspondentes em `dim_listings`| 58.805  |
| Listings sem correspondência (órfãos)     | 0       |

### Interpretação

Os resultados mostram **integridade referencial perfeita** entre a tabela de fatos (`fact_reviews`) e a dimensão de anúncios (`dim_listings`):

- Todos os `listing_id` presentes em `fact_reviews` existem em `dim_listings`.  
- Não há qualquer review associado a um anúncio inexistente.  
- A carga e as transformações nas camadas Bronze → Silver → Gold preservaram o relacionamento corretamente.  

Isso é essencial para garantir:

- consistência na modelagem dimensional,  
- confiabilidade das análises de impacto de reviews na performance,  
- segurança ao construir métricas como *reviews por mês*, *review recency*, *rating médio*, etc.

A ausência de reviews órfãos é um forte indicador de que o pipeline foi implementado corretamente e sem perda de chaves.


#Conclusões Finais da Análise de Qualidade dos Dados

A avaliação sistemática das camadas Gold (dimensões e fatos) demonstrou que o dataset do Inside Airbnb, após o pipeline Bronze → Silver → Gold, apresenta **alta qualidade estrutural**, **coerência lógica** e **excelente integridade referencial**.

As conclusões principais estão organizadas por tema:

---

##Completude

### Dimensões
- `dim_listings` possui chaves completas (sem nulos em `listing_id` ou `host_id`).
- Campos críticos como `room_type`, `accommodates` e `neighbourhood_cleansed` apresentam **zero nulos**.
- O único campo com lacuna relevante é `price_numeric` (24,22% nulos), que está documentado e explicado.

###Fatos
- `fact_calendar` tem nulo apenas nas colunas herdadas do calendário original (preço), o que é esperado e não compromete ocupação.
- `fact_reviews` é **100% completa** em todos os campos essenciais.

---

##Integridade Referencial

Todos os testes confirmam que:

- **Cada review tem correspondência em `dim_listings`**
- **Cada registro de calendário tem correspondência em `dim_listings`**
- **Cada host utilizado em listings consta em `dim_hosts`**

Não foram encontrados:

- reviews órfãos  
- calendários órfãos  
- hosts sem listings  
- listings sem bairro válido  

*Resultado: excelente integridade dimensional.*

---

## Coerência Lógica

Os testes mostraram:

- `available_bool` e `is_occupied` apresentam **zero contradições** em mais de 28 milhões de registros.
- Eventos de disponibilidade seguem a semântica original do Airbnb.
- Não existem datas inválidas ou campos temporalmente incoerentes nas dimensões.

---

##Semântica de Negócio

A principal lacuna - anúncios com `price_numeric` nulo - foi estudada em profundidade e validada:

- Ainda aparecem no calendário (portanto, **ativos**)
- 1.636 possuem reviews nos últimos 6 meses (**ativos de fato**)
- 5.909 não têm reviews (possivelmente pouco atrativos / pouco utilizados)
- Todos possuem `estimated_occupancy` disponível no dataset (Airbnb fornece estimativas internas mesmo sem preço explícito)

Conclusão:

> A ausência do preço não compromete as análises do MVP, desde que tratada adequadamente nas métricas que dependem de valores monetários.

---

##Qualidade dos Dados - Resumo

| Tabela              | Completude | Integridade | Coerência Lógica | Situação |
|--------------------|------------|-------------|-------------------|----------|
| `dim_listings`     | Alta (exceto preço) | Perfeita | Perfeita | Aprovada |
| `dim_hosts`        | Alta | Perfeita | Perfeita | Aprovada |
| `dim_neighbourhoods` | Alta | Perfeita | Não aplicável | Aprovada |
| `fact_calendar`    | Média (preço nulo por design) | Perfeita | Perfeita | Aprovada com ressalvas documentadas |
| `fact_reviews`     | Excelente | Perfeita | Perfeita | Aprovada |

---

#Conclusão Geral

A camada Gold é **confiável**, **consistente** e **adequada para análises exploratórias e construção de insights**.

O dataset não apresenta problemas estruturais que comprometam o MVP, e todos os fenômenos observados (como ausência de preço diário) estão explicados e documentados conforme a realidade do dataset Inside Airbnb.

Assim, estamos prontos para iniciar a etapa seguinte:

> **Análise Exploratória Gold - entendimento de comportamento de mercado, preços, ocupação e performance.**
